# Final Round - KTBG TechJam Data Track by Team Watchara

## This is my step-by-step Guide of how I build my model

### Import the libraries

In [1]:
import pandas as pd
import numpy as np
import math
from pandas.tseries.offsets import MonthEnd
from dateutil.relativedelta import relativedelta, FR
import datetime
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_absolute_error, accuracy_score, classification_report
from lightgbm import LGBMRegressor, LGBMClassifier
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler

/home/aiy/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


### Selected Categories required by the Question

In [2]:
SELECTED_CAT = ['Automobiles and Vehicles', 'Clothing Stores', 'Service Providers',
            'Transportation', 'Utilities']

### Import the Data - 3 files are given

In [3]:
ccinfo = pd.read_csv('data/cc_info.csv.zip', index_col='card_no') #, parse_dates=['txn_date'])
cclog = pd.read_csv('data/cc_log.csv.zip', parse_dates=['txn_dt'])
cat_map = pd.read_csv('data/Final_categories.csv') #, parse_dates=['txn_date'])

### Explore the data

In [4]:
ccinfo.head()

,card_type,opn_dt,exp_dt,cr_lmt_amt,prev_cr_lmt_amt,main_zip_cd,cr_line_amt,incm_amt,brth_estb_yr,gnd_ind
card_no,,,,,,,,,,
4410123456000001,visa,1997-10-14,1017,50000,0,73120.0,50000,21000,1953.0,0
4410123456000002,master,2010-06-29,620,146000,0,43000.0,146000,72000,1965.0,0
4410123456000003,visa,2014-06-18,619,22000,0,23000.0,22000,20000,1978.0,0
4410123456000004,visa,2014-11-03,1119,1000000,0,10200.0,2141000,1800000,1955.0,1
4410123456000005,visa,2014-04-10,419,80000,0,57160.0,80000,32000,1969.0,0


In [5]:
cclog.head()

,card_no,txn_dt,txn_tm,bill_amt,card_acpt_cty,mrch_tp_cd,card_type
0,4410123456098153,2017-03-20 17:00:00,10:22:19,5660,TH,4722,visa
1,4410123456018545,2017-03-20 17:00:00,23:30:43,100,US,5818,visa
2,4410123456085556,2017-03-20 17:00:00,07:32:54,1540,TH,6300,visa
3,4410123456047294,2017-03-20 17:00:00,05:16:20,1500,TH,5541,visa
4,4410123456074332,2017-03-20 17:00:00,13:18:56,680,TH,5812,visa


In [6]:
cat_map.head()

,Categories,MCC
0,Airlines,3000
1,Airlines,3001
2,Airlines,3002
3,Airlines,3003
4,Airlines,3004


### Select only Category codes required by the question

In [7]:
cat_map['Categories'].unique()

array(['Airlines', 'Amusement and Entertainment',
       'Automobile/Vehicle Rentals', 'Automobiles and Vehicles',
       'Business Services', 'Clothing Stores', 'Contracted Services',
       'Government Services', 'Hotels and Motels',
       'Mail Order/Telephone Order Providers', 'Miscellaneous Stores',
       'Personal Service Providers',
       'Professional Services and Membership Organizations',
       'Repair Services', 'Retail Stores', 'Service Providers',
       'Transportation', 'Utilities',
       'Wholesale Distributors and Manufacturers'], dtype=object)

In [8]:
selected_cat_id = cat_map[cat_map['Categories'].isin(SELECTED_CAT)]

In [9]:
selected_cat_id['Categories'].unique()

array(['Automobiles and Vehicles', 'Clothing Stores', 'Service Providers',
       'Transportation', 'Utilities'], dtype=object)

### Drop some columns that not very relevant

In [10]:
cclog.drop(['txn_tm', 'card_acpt_cty', 'card_type'], axis=1, inplace=True)
ccinfo.drop(['card_type', 'opn_dt', 'exp_dt', 'main_zip_cd'], axis=1, inplace=True)

### extract month, year, and order the transactions by month with 0 for the first month of the transactions and 24 for the last month

In [11]:
cclog['month'] = cclog['txn_dt'].dt.month
cclog['year'] = cclog['txn_dt'].dt.year

cclog['month_rank'] = 0
cclog.loc[cclog['year'] == 2016, 'month_rank'] = cclog.loc[cclog['year'] == 2016, 'month']
cclog.loc[cclog['year'] == 2017, 'month_rank'] = cclog.loc[cclog['year'] == 2017, 'month'] + 12

In [12]:
cclog.head()

,card_no,txn_dt,bill_amt,mrch_tp_cd,month,year,month_rank
0,4410123456098153,2017-03-20 17:00:00,5660,4722,3,2017,15
1,4410123456018545,2017-03-20 17:00:00,100,5818,3,2017,15
2,4410123456085556,2017-03-20 17:00:00,1540,6300,3,2017,15
3,4410123456047294,2017-03-20 17:00:00,1500,5541,3,2017,15
4,4410123456074332,2017-03-20 17:00:00,680,5812,3,2017,15


### Select only transactions required by the question

In [13]:
cclog_filter_cat = cclog[cclog['mrch_tp_cd'].isin(selected_cat_id['MCC'].values)]

### add category name to the dataframe

In [14]:
cclog_filter_cat['cat_name'] = ''

for i, cat_name in enumerate(SELECTED_CAT):
    temp = selected_cat_id[selected_cat_id['Categories'] == cat_name]
    select_row = cclog_filter_cat['mrch_tp_cd'].isin(temp['MCC'].values)
    cclog_filter_cat.loc[select_row, 'cat_name'] = cat_name

cclog_filter_cat.head()

/home/aiy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/aiy/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,card_no,txn_dt,bill_amt,mrch_tp_cd,month,year,month_rank,cat_name
0,4410123456098153,2017-03-20 17:00:00,5660,4722,3,2017,15,Transportation
2,4410123456085556,2017-03-20 17:00:00,1540,6300,3,2017,15,Service Providers
3,4410123456047294,2017-03-20 17:00:00,1500,5541,3,2017,15,Automobiles and Vehicles
6,4410123456008859,2017-03-20 17:00:00,1000,5631,3,2017,15,Clothing Stores
7,4410123456021078,2017-03-20 17:00:00,80,4121,3,2017,15,Transportation


### Find How many days from last used for each card and categories

In [15]:
sort_time = cclog_filter_cat.sort_values(['card_no', 'cat_name', 'txn_dt'],ascending=False).groupby(['card_no', 'cat_name']).head(1)
sort_time['days_from_last'] = (datetime.datetime.now() - sort_time['txn_dt']).dt.days

In [16]:
sort_time.head()

,card_no,txn_dt,bill_amt,mrch_tp_cd,month,year,month_rank,cat_name,days_from_last
5706786,4410123456100000,2016-08-15 17:00:00,4000,4812,8,2016,8,Utilities,399
2290947,4410123456100000,2016-04-02 17:00:00,3000,6011,4,2016,4,Service Providers,534
1210327,4410123456100000,2016-04-02 17:00:00,1180,5699,4,2016,4,Clothing Stores,534
4939788,4410123456100000,2017-06-27 17:00:00,580,5541,6,2017,18,Automobiles and Vehicles,83
5972452,4410123456099999,2017-05-20 17:00:00,860,4812,5,2017,17,Utilities,121


### Find sum amount spent and number of transactions for each categories and card

In [17]:
month_group = cclog_filter_cat.groupby(['cat_name', 'month_rank', 'card_no'])

group_sum = month_group.sum()[['bill_amt']]
group_count = month_group.count()[['txn_dt']]

group_log = pd.merge(group_sum, group_count, how='outer',
                          left_index=True, right_index=True)

group_log.head()

bill_amt  txn_dt
cat_name                 month_rank card_no                           
Automobiles and Vehicles 0          4410123456000117      1060       1
                                    4410123456000300      1380       1
                                    4410123456000327       700       1
                                    4410123456000352       500       1
                                    4410123456000520      1260       1

### filter only transactions with month_rank 14+ and sum, count by categories and card_no

In [18]:
cclog_filter_cat_10up = cclog_filter_cat[cclog_filter_cat['month_rank'] > 13]
card_group = cclog_filter_cat_10up.groupby(['cat_name', 'card_no'])
group_card_sum = card_group.sum()[['bill_amt']]
group_card_count = card_group.count()[['txn_dt']]
group_agg = card_group.agg(['mean', 'count'])[['bill_amt']]

group_log_card = pd.merge(group_card_sum, group_card_count, how='outer',
                          left_index=True, right_index=True)
group_log_card.head()

bill_amt  txn_dt
cat_name                 card_no                           
Automobiles and Vehicles 4410123456000003      5680       5
                         4410123456000004      3900       3
                         4410123456000005      8680      11
                         4410123456000007     26980      23
                         4410123456000009      4520       3

### Find sum, count by months

In [19]:
all_spend = pd.DataFrame()
for cat_name in SELECTED_CAT:
    temp = pd.DataFrame(index=ccinfo.index)
    temp['total_count'] = 0
    temp['total_spend'] = 0
    temp['mean'] = 0
    for month_rank in range(19):
        card_sum = group_log.loc[(cat_name, month_rank)]
        temp[['bill'+str(month_rank), 'count'+str(month_rank)]] = card_sum
        
    temp[['total_spend', 'total_count']] = group_log_card.loc[(cat_name)]
    temp['mean'] = group_agg.loc[(cat_name), ('bill_amt','mean')]
        
    sort_time_temp = sort_time[sort_time['cat_name'] == cat_name].set_index('card_no')
    temp['days_from_last'] = sort_time_temp['days_from_last']
    
    temp['cat_name'] = cat_name
    all_spend = all_spend.append(temp.reset_index(), ignore_index=True)
    
all_spend.fillna(0, inplace=True)

In [20]:
all_spend.head()

,card_no,total_count,total_spend,mean,bill0,count0,bill1,count1,bill2,count2,...,bill15,count15,bill16,count16,bill17,count17,bill18,count18,days_from_last,cat_name
0,4410123456000001,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,391.0,Automobiles and Vehicles
1,4410123456000002,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Automobiles and Vehicles
2,4410123456000003,5.0,5680.0,1136.000000,0.0,0.0,0.0,0.0,500.0,1.0,...,0.0,0.0,840.0,1.0,0.0,0.0,0.0,0.0,160.0,Automobiles and Vehicles
3,4410123456000004,3.0,3900.0,1300.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2400.0,2.0,0.0,0.0,1500.0,1.0,101.0,Automobiles and Vehicles
4,4410123456000005,11.0,8680.0,789.090909,0.0,0.0,0.0,0.0,0.0,0.0,...,920.0,1.0,2640.0,3.0,2240.0,3.0,1940.0,3.0,92.0,Automobiles and Vehicles


### label encode cat_name

In [21]:
all_spend['cat_int'] = all_spend['cat_name'].apply(lambda x: SELECTED_CAT.index(x))
all_spend['cat_int'].unique()

array([0, 1, 2, 3, 4])

### Merge every things together and select features

In [23]:
dataset = pd.merge(ccinfo, all_spend, how='right',
                   left_index=True, right_on='card_no')

## Start Modeling

### 1st model for classifying 0s and non 0s

In [24]:
X = dataset[['total_count', 'brth_estb_yr', 'gnd_ind', 'mean', 'bill6', 'count6', 'bill15', 'count15',
             'bill16', 'count16', 'bill17', 'count17', 'cat_int']]

X = pd.get_dummies(X, columns = ['cat_int'] ).values
y = dataset['count18'].values >= 1

from xgboost import XGBClassifier
model_class = XGBClassifier(min_child_weight=10, max_depth=3, gamma=1,
                           reg_alpha=0.015)

model_class.fit(X,y)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=1, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=10, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0.015, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

### 2nd Model - Predict Count (How many times card used next month) - only those with predicted non 0s values from 1st model selected

In [26]:
dataset_reg = dataset[dataset['count18'] > 0]


X = dataset_reg[['total_count', 'gnd_ind', 'mean', 'bill6', 'count6', 'bill15', 'count15',
             'bill16', 'count16', 'bill17', 'count17', 'cat_int']]

X = pd.get_dummies(X, columns = ['cat_int'] ).values
y = dataset_reg['count18']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

model_reg_f = LGBMRegressor(n_estimators=100, num_leaves=80)
model_reg_f.fit(X, y) 

y_pred = model_reg_f.predict(X_test).round()
mean_absolute_error(y_test, y_pred)

0.43716663110731813

### 3rd model - Predicting amount spent next month

In [27]:
dataset_reg = dataset[dataset['count18'] > 0]

X = dataset_reg[['total_count', 'gnd_ind', 'mean', 'bill6', 'count6', 'bill15', 'count15',
             'bill16', 'count16', 'bill17', 'count17', 'cat_int']]

X = pd.get_dummies(X, columns = ['cat_int'] ).values
y = dataset_reg['bill18']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


model_reg_s = LGBMRegressor(n_estimators=400, num_leaves=80, learning_rate=0.05)
model_reg_s.fit(X, y)

y_pred = model_reg_s.predict(X_test)
mean_absolute_error(y_test, y_pred)

8280.7117459418914

## Predicting the answer

### Using 1st Model

In [28]:
dataset.sort_values(['card_no', 'cat_int'], inplace=True)
dataset.reset_index(drop=True, inplace=True)


X = dataset[['total_count', 'brth_estb_yr', 'gnd_ind', 'mean', 'bill7', 'count7', 'bill16', 'count16',
             'bill17', 'count17', 'bill18', 'count18', 'cat_int']]

X = pd.get_dummies(X, columns = ['cat_int'] ).values

y_buy = model_class.predict_proba(X)[:,1] > 0.1

### Using 2nd Model

In [29]:
dataset_buy = dataset[y_buy]

X = dataset_buy[['total_count', 'gnd_ind', 'mean', 'bill7', 'count7', 'bill16', 'count16',
             'bill17', 'count17', 'bill18', 'count18', 'cat_int']]

X = pd.get_dummies(X, columns = ['cat_int'] ).values

y_pred_f = model_reg_f.predict(X).round()

### Using 3rd Model

In [30]:
y_pred_s = model_reg_s.predict(X).round()

## Format the answer for submitting

In [31]:
dataset.loc[y_buy, 'count_ans'] = y_pred_f
dataset.loc[y_buy, 'spend_ans'] = y_pred_s

dataset.fillna(0, inplace=True)

ans = dataset[['count_ans', 'spend_ans']]

ans_df = pd.DataFrame({'auto_f': ans['count_ans'][::5].reset_index(drop=True), 'cloth_f': ans['count_ans'][1::5].reset_index(drop=True), 
                        'serv_f': ans['count_ans'][2::5].reset_index(drop=True), 'tran_f': ans['count_ans'][3::5].reset_index(drop=True), 
                        'util_f': ans['count_ans'][4::5].reset_index(drop=True), 'auto_s': ans['spend_ans'][::5].reset_index(drop=True), 
                        'cloth_s': ans['spend_ans'][1::5].reset_index(drop=True), 'serv_s': ans['spend_ans'][2::5].reset_index(drop=True), 
                        'tran_s': ans['spend_ans'][3::5].reset_index(drop=True), 'util_s': ans['spend_ans'][4::5].reset_index(drop=True), })

ans_df = ans_df[['auto_f', 'cloth_f', 'serv_f', 'tran_f', 'util_f',
                   'auto_s', 'cloth_s', 'serv_s', 'tran_s', 'util_s']]

ans_df.to_csv('Team_19.csv', header=False, index=False, sep=',')